Expected Value vs. CVaR Comparison

Mauricio Tombini, Jordan Jalving, and Victor M. Zavala 

University of Wisconsin-Madison, 2016

In [1]:
# Loading packages:
using JuMP 
using Distributions 
using Ipopt
using PyPlot

INFO: Precompiling module PyPlot...

signal (11): Segmentation fault: 11
PyObject_Call at /System/Library/Frameworks/Python.framework/Versions/2.7/Python (unknown line)
unknown function (ip: 0x320905760)
unknown function (ip: 0x32090457e)
unknown function (ip: 0x3209040f3)
unknown function (ip: 0x320904026)
jl_apply_generic at /opt/local/lib/julia/libjulia.dylib (unknown line)
unknown function (ip: 0x3208c2178)
unknown function (ip: 0x3208c2020)
unknown function (ip: 0x3208a5249)
unknown function (ip: 0x3208a5020)
unknown function (ip: 0x3208a31e6)
unknown function (ip: 0x3208a3055)
unknown function (ip: 0x3208a301b)
jl_apply_generic at /opt/local/lib/julia/libjulia.dylib (unknown line)
unknown function (ip: 0x32089d0d6)
unknown function (ip: 0x32089b28e)
jl_apply_generic at /opt/local/lib/julia/libjulia.dylib (unknown line)
unknown function (ip: 0x32089a0d6)
jl_apply_generic at /opt/local/lib/julia/libjulia.dylib (unknown line)
_start at /opt/local/lib/julia/sys.dylib (unknown line)
j

LoadError: LoadError: Failed to precompile PyPlot to /Users/vzavala/.julia/lib/v0.4/PyPlot.ji
while loading In[1], in expression starting on line 5

In [2]:
# Generate random data: 
NS = 100
S = collect(1:NS); 
srand(0)
μ = 0; σ = 2; 
d = Normal(μ,σ)
R = rand(d,NS);

In [3]:
# Plotting data
plt[:hist](R, bins = 30);
grid("on")
xlabel(L"\xi")
ylabel(L"p(\xi)")

LoadError: LoadError: UndefVarError: plt not defined
while loading In[3], in expression starting on line 2

## Solve expected value problem

In [4]:
m = Model(solver=IpoptSolver(print_level=0))

@variable(m, x)            # decision variable
@variable(m, cost[1:NS])   # per scenario cost
@constraint(m, costeq[s in S], cost[s] == (x-R[s])^2) 
@objective(m, Min, (1/NS)*sum{cost[s], s in S})

solve(m)
solcost = getvalue(cost);
println("x=",getvalue(x))
println("mean=",getobjectivevalue(m))
println("meanR=",(1/NS)*sum(R))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

x=-0.049640963099936354
mean=4.41838656225072
meanR=-0.04964096309993606


In [5]:
# Plotting cost fistribution 
plt[:hist](solcost,bins = 30)
grid("on")
xlabel(L"f(x^*(\xi),\xi)")
ylabel(L"p(f(\cdot))")

LoadError: LoadError: UndefVarError: plt not defined
while loading In[5], in expression starting on line 2

## Solve CVaR Problem

In [6]:
m = Model(solver=IpoptSolver(print_level=0))

@variable(m, x)              # decision variable
@variable(m, cost[1:NS])     # per scenario cost
@variable(m, VaR)            # cvar auxiliary variable
@variable(m, phi[S] >= 0)    # cvar auxiliary variable
alpha = 0.0001;              # cvar probability level

@constraint(m, costeq[s in S], cost[s] == (x-R[s])^2) 
@constraint(m, cvar[s in S], cost[s]-VaR <= phi[s])
@objective(m, Min, VaR + (1/NS)*sum{(1/alpha)*phi[s], s in S})

solve(m)
solcost2 = getvalue(cost);
println("x=",getvalue(x))
println("CVaR=",getobjectivevalue(m))
println("mean=",(1/NS)*sum(solcost2))

x=-1.153574009577105
CVaR=33.42170024257977
mean=5.637054734120719


In [7]:
# Plotting cost distribution 
plt[:hist](solcost2,bins = 30)
grid("on")
xlabel(L"f(x^*(\xi),\xi)")
ylabel(L"p(f(\cdot))")

LoadError: LoadError: UndefVarError: plt not defined
while loading In[7], in expression starting on line 2